In [1]:
!pip install indic_transliteration indicsyllabifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 4.1 MB/s 
  Created wheel for indicsyllabifier: filename=indicsyllabifier-0.3-py3-none-any.whl size=6097 sha256=2ddb3e5e53dcce6aff6e0c5067231dfb13ee9931ef87d31d648f797783ddac30
  Stored in directory: /root/.cache/pip/wheels/8d/4f/20/83c943b8c9f1523fafacfd787390cba787195f5158da262644
  Created wheel for silpa-common: filename=silpa_common-0.3-py3-none-any.whl size=8481 sha256=53743d12cde54ae114d177f07e26cd46a19b0ec9abea71e6e70e2f193bdd26f7
  Stored in directory: /root/.cache/pip/wheels/c7/a5/67/816bbc59aef9be86b1d58f1db723f8b5896ee9605ea0603b5b
Successfully built indicsyllabifier silpa-common


In [2]:
from indic_transliteration import detect, sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import indicsyllabifier
from random import choice,sample
from itertools import combinations

In [3]:
def Split_Number(number):  
    vowels={'a':1,'A':1,'i':10**2,'I':10**2,'u':10**4,'U':10**4,'f':10**6,'F':10**6,'x':10**8,'X':10**8,'e':10**10,'E':10**12,'o':10**14,'O':10**16}
    consonants={'k':1,'K':2,'g':3,'G':4,'N':5,'c':6,'C':7,'j':8,'J':9,'Y':10,'w':11,'W':12,'q':13,'Q':14,'R':15,'t':16,'T':17,'d':18,'D':19,'n':20,'p':21,'P':22,'b':23,'B':24,'m':25,'y':30,'r':40,'l':50,'v':60,'S':70,'z':80,'s':90,'h':100}
    consonants_keys = list(consonants.keys())
    consonants_values = list(consonants.values())
    digits=[]
    if number<=100:
      digits.append(number)
    else:
      while number>100:
        d=number%100
        digits.append(int(d))
        number=(number-d)/100
        if number <=100:
          digits.append(int(number))
    print(digits)

    split_digits=[]
    for digit in digits:
      a=[]
      i=1
      while i<digit:
        a.append(i)
        i=i+1
      b=[]
      if digit==0:
        b.append([0])
      if digit in consonants.values():
        b.append([digit])
      if digit>2:  
        comb=combinations(a,2)
        for x in list(comb):
          c=[]
          i=0
          while i<2:
            if i==0:
              if  x[i] in consonants.values():
                c.append(x[i])
            else:
              if (x[i]-x[i-1]) in consonants.values():   
                c.append(x[i]-x[i-1])
              if (digit-x[i]) in consonants.values():  
                c.append(digit-x[i])
            i=i+1 
          if len(c)==3:
            b.append(c)
      if digit>1:  
        comb=combinations(a,1)
        for x in list(comb):
          c=[]
          if x[0] in consonants.values() and digit-x[0] in consonants.values():
            c.append(x[0])
            c.append(digit-x[0])
            b.append(c)
      split_digits.append(b)
    return split_digits  


In [4]:
def Aryabhatan_Encoder(number,show):
    vowels={'a':1,'A':1,'i':10**2,'I':10**2,'u':10**4,'U':10**4,'f':10**6,'F':10**6,'x':10**8,'X':10**8,'e':10**10,'E':10**12,'o':10**14,'O':10**16}
    consonants={'k':1,'K':2,'g':3,'G':4,'N':5,'c':6,'C':7,'j':8,'J':9,'Y':10,'w':11,'W':12,'q':13,'Q':14,'R':15,'t':16,'T':17,'d':18,'D':19,'n':20,'p':21,'P':22,'b':23,'B':24,'m':25,'y':30,'r':40,'l':50,'v':60,'S':70,'z':80,'s':90,'h':100}
    consonants_keys = list(consonants.keys())
    consonants_values = list(consonants.values())
    split_digits=Split_Number(number)
    syllable_collection=[]    
    for b in split_digits:
      syllables=[]
      for c in b:
        if c==[0]:
          syllables.append("")   
        else:  
          syllable=""
          for x in c:
            syllable=syllable+consonants_keys[consonants_values.index(x)]   
          syllables.append(syllable)          
      syllable_collection.append(syllables)
    #for x in syllable_collection:
      #print(f"\n {x}")  

    total_words_possible=1
    for syllables in syllable_collection:
      #print(f"{len(syllables)}, {total_words_possible}")
      total_words_possible=total_words_possible*len(syllables)
      #print(f"new {total_words_possible}")  

    #print(total_words_possible)
    word_collection=[]
    while len(word_collection)<show and len(word_collection)<total_words_possible:
      word=""
      i=0
      for x in syllable_collection: 
        if x==[""]:
          i=i+2
        else:  
          syllable=choice(x)
          a=[]
          for x,y in vowels.items():
            #print(10**i)
            if y==10**i:
              a.append(x)
          #print(a)
          word=word+syllable+choice(a)
          i=i+2
      if word not in word_collection:
        word_collection.append(word)
    return ( word_collection ,total_words_possible )  


In [5]:
def main():
    print("\nEnter number")
    number=int(input())
    print("\nHow many words do you want to see ")
    show=int(input())
    #print("\n")
    word_collection, total_words_possible=Aryabhatan_Encoder(number,show)
    s1 = SchemeMap(SCHEMES[sanscript.SLP1], SCHEMES[sanscript.DEVANAGARI])
    if len(word_collection)==show:
      i=1
      print("\n")
      for word in word_collection:  
        print(f"{i}. {transliterate(word,scheme_map=s1)}")
        i=i+1
    else:
      print(f"\nOnly {total_words_possible} words are available \n")
      i=1
      for word in word_collection:  
        print(f"{i}. {transliterate(word,scheme_map=s1)}")
        i=i+1    


In [ ]:
main()


Enter number
5264


In [ ]:
main()


Enter number
400

How many words do you want to see 
45

Only 0 words are available 



In [ ]:
#def Aryabhatan_Encoder(number, show):  
    vowels={'a':1,'A':1,'i':10**2,'I':10**2,'u':10**4,'U':10**4,'f':10**6,'F':10**6,'x':10**8,'X':10**8,'e':10**10,'E':10**12,'o':10**14,'O':10**16}
    consonants={'k':1,'K':2,'g':3,'G':4,'N':5,'c':6,'C':7,'j':8,'J':9,'Y':10,'w':11,'W':12,'q':13,'Q':14,'R':15,'t':16,'T':17,'d':18,'D':19,'n':20,'p':21,'P':22,'b':23,'B':24,'m':25,'y':30,'r':40,'l':50,'v':60,'S':70,'z':80,'s':90,'h':100}
    consonants_keys = list(consonants.keys())
    consonants_values = list(consonants.values())
    digits=[]
    while number>0:
      d=number%100
      digits.append(int(d))
      number=(number-d)/100
    #print(digits)

    split_digits=[]
    for digit in digits:
      a=[]
      i=1
      while i<digit:
        a.append(i)
        i=i+1
      b=[]
      if digit==0:
        b.append([0])
      if digit in consonants.values():
        b.append([digit])
      if digit>2:  
        comb=combinations(a,2)
        for x in list(comb):
          c=[]
          i=0
          while i<2:
            if i==0:
              if  x[i] in consonants.values():
                c.append(x[i])
            else:
              if (x[i]-x[i-1]) in consonants.values():   
                c.append(x[i]-x[i-1])
              if (digit-x[i]) in consonants.values():  
                c.append(digit-x[i])
            i=i+1 
          if len(c)==3:
            b.append(c)
      if digit>1:  
        comb=combinations(a,1)
        for x in list(comb):
          c=[]
          if x[0] in consonants.values() and digit-x[0] in consonants.values():
            c.append(x[0])
            c.append(digit-x[0])
            b.append(c)
      split_digits.append(b)

    syllable_collection=[]    
    for b in split_digits:
      syllables=[]
      for c in b:
        if c==[0]:
          syllables.append("")   
        else:  
          syllable=""
          for x in c:
            syllable=syllable+consonants_keys[consonants_values.index(x)]   
          syllables.append(syllable)          
      syllable_collection.append(syllables)
    #for x in syllable_collection:
      #print(f"\n {x}")  

    total_words_possible=1
    for syllables in syllable_collection:
      #print(f"{len(syllables)}, {total_words_possible}")
      total_words_possible=total_words_possible*len(syllables)
      #print(f"new {total_words_possible}")  

    #print(total_words_possible)
    word_collection=[]
    while len(word_collection)<show and len(word_collection)<total_words_possible:
      word=""
      i=0
      for x in syllable_collection: 
        if x==[""]:
          i=i+2
        else:  
          syllable=choice(x)
          a=[]
          for x,y in vowels.items():
            #print(10**i)
            if y==10**i:
              a.append(x)
          #print(a)
          word=word+syllable+choice(a)
          i=i+2
      if word not in word_collection:
        word_collection.append(word)
    return ( word_collection ,total_words_possible ) 
        

        
    
           


In [ ]:
a=[]
i=0
while i<900:
  a.append(i)
  i=i+1
comb=combinations(a,2)
print(list(comb))  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

